In [4]:
import pandas as pd

In [6]:
df=pd.read_csv('data/data.csv')
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [7]:
df.drop(columns=['Order_Date','ID','Restaurant_latitude','Restaurant_longitude','Delivery_person_ID'],axis=1,inplace=True)

In [8]:
def str_min(x):
    min=0
    if type(x)==list:
        if len(x)>1:
            min = x[1]
        else:
            min = int(float(x[0])*60)
    return int(min)

In [9]:
df['Time_Orderd_min'] = df['Time_Orderd'].fillna(df['Time_Orderd'].mode()[0]).str.split(':').apply(str_min)
df['Time_Order_picked_min'] = df['Time_Order_picked'].fillna(df['Time_Orderd'].mode()[0]).str.split(':').apply(str_min)
df.drop(columns=['Time_Orderd','Time_Order_picked'],axis=1,inplace=True)

In [10]:
#Calculating time difference order and pickup and removing the minutes column
def Time_diff_bw_order_and_pickup():
    x = []
    for i in range(len(df)):
        if df['Time_Order_picked_min'][i] < df['Time_Orderd_min'][i]:
            x.append(df['Time_Order_picked_min'][i] + (60-df['Time_Orderd_min'][i]))
        else:
            x.append(df['Time_Order_picked_min'][i] - df['Time_Orderd_min'][i])
    df['Time_diff_bw_order_and_pickup'] = x
    df.drop(columns=['Time_Orderd_min','Time_Order_picked_min'],axis=1,inplace=True)

In [11]:
Time_diff_bw_order_and_pickup()

In [12]:
X = df.drop(columns=['Time_taken (min)'],axis=1)
y=df['Time_taken (min)']

In [13]:
#Segregate numerical columns and categorical columns
numerical_columns = X.columns[X.dtypes!='object']
categorical_columns = X.columns[X.dtypes=='object']
print('numerical_columns',numerical_columns)
print('categorical_columns',categorical_columns)

numerical_columns Index(['Delivery_person_Age', 'Delivery_person_Ratings',
       'Delivery_location_latitude', 'Delivery_location_longitude',
       'Vehicle_condition', 'multiple_deliveries',
       'Time_diff_bw_order_and_pickup'],
      dtype='object')
categorical_columns Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City'],
      dtype='object')


In [14]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [15]:
categories = ['Fog','Stormy','Sandstorms','Windy','Cloudy','Sunny'],['Jam', 'High', 'Medium', 'Low'],['Drinks','Snack','Meal','Buffet'],['bicycle','electric_scooter','scooter','motorcycle'],['Yes','No'],['Semi-Urban','Urban','Metropolitian']
# ordinal_cols = df[categorical_columns].drop(columns=,axis=1)

In [16]:
#Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
    ]
)

#Categorical pipeline
cat_pipeline = Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OrdinalEncoder(categories=list(categories))),
    ('scaler',StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('numerical pipeline', num_pipeline,numerical_columns),
    ('categorical pipeline',cat_pipeline,categorical_columns)
])

In [17]:
#train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.3,random_state=42)

In [18]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.fit_transform(X_test),columns=preprocessor.get_feature_names_out())

In [19]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.svm import SVR
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [21]:
import numpy as np
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    R2Score = r2_score(true,predicted)
    return mae,mse,R2Score

In [22]:
models = {
    'Linear Regression': LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet(),
    'SVR':SVR()
}
trained_model_list = []
model_list = []
r2_list = []
for i in range(len(list(models))):
    model= list(models.values())[i]
    model.fit(X_train,y_train)

    #MAke predictions
    y_pred = model.predict(X_test)

    mae,mse,r2 = evaluate_model(y_test,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Mean Squared Error', mse)
    print('Mean Absolute Error', mae)
    print('R2 score',r2*100)

    r2_list.append(r2_score)

    print('='*100)

Linear Regression
Mean Squared Error 44.89516381283955
Mean Absolute Error 5.294126832426518
R2 score 49.7432599908969
Lasso
Mean Squared Error 51.00534345570005
Mean Absolute Error 5.714738793655433
R2 score 42.903376055952144
Ridge
Mean Squared Error 44.8951693954732
Mean Absolute Error 5.294128349569822
R2 score 49.743253741560814
ElasticNet
Mean Squared Error 51.984671971432505
Mean Absolute Error 5.788392481638126
R2 score 41.807091859198856
SVR
Mean Squared Error 36.18487271219909
Mean Absolute Error 4.674594888127322
R2 score 59.49377202986401


In [ ]:
model_list

['Linear Regression', 'Lasso', 'Ridge', 'ElasticNet', 'SVR']